# RAG with Unstructured & AstraDB

This example shows loading and parsing a PDF document with Unstructured.io into an Astra DB Serverless vector store, then querying the index with LangChain.

### Requirements

In [1]:
! pip install --quiet ragstack-ai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 22.9 M

### Configuration

To use Unstructured.io, you need an API key. Sign-up for one here: https://unstructured.io/api-key-hosted.

In [2]:
import os
from getpass import getpass

os.environ["UNSTRUCTURED_API_KEY"] = getpass("Enter your Unstructured API Key:")
os.environ["UNSTRUCTURED_API_URL"] = getpass("Enter your Unstructured API URL:")
os.environ["ASTRA_DB_API_ENDPOINT"] = input("Enter your Astra DB API Endpoint: ")
os.environ["ASTRA_DB_APPLICATION_TOKEN"] = getpass("Enter your Astra DB Token: ")
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key: ")


Enter your Unstructured API Key:··········
Enter your Unstructured API URL:··········
Enter your Astra DB API Endpoint: https://756b996f-a063-4881-9757-1f7299209ae5-us-east-1.apps.astra.datastax.com
Enter your Astra DB Token: ··········
Enter your OpenAI API Key: ··········


### Using the Unstructured API to parse a PDF

#### Advanced Parsing

The unstructured library aims to simplify and streamline the preprocessing of structured and unstructured documents for downstream tasks. When we partition a document, the output is a list of document Element objects. These element objects represent different components of the source document. Currently, the unstructured library supports the following element types:

* type
    * FigureCaption
    * NarrativeText
    * ListItem
    * Title
    * Address
    * Table
    * PageBreak
    * Header
    * Footer
    * UncategorizedText
    * Image
    * Formula
* element_id
* metadata - see: Metadata page
* text



In [33]:
from langchain_community.document_loaders import unstructured

elements = unstructured.get_elements_from_api(
    file_path="/content/ES-20.02-D2_Quality_Requirement_for_Pressure_Vessel.pdf",
    api_key=os.getenv("UNSTRUCTURED_API_KEY"),
    api_url = os.getenv("UNSTRUCTURED_API_URL"),
    strategy="hi_res", # default "auto"
    pdf_infer_table_structure=True,
)

len(elements)

395

In [34]:
from IPython.display import display, HTML

parents = {}

for el in elements:
    parents[el.id] = el.text

for el in elements:
    if el.category == "Table":
        display(HTML(el.metadata.text_as_html))
    elif el.metadata.parent_id:
        print(f"parent: '{parents[el.metadata.parent_id]}' content: {el.text}")
    else:
        print(el)

QUALITY REQUIREMENTS FOR PRESSURE VESSEL
  
PTT PUBLIC CO., LTD ENGINEERING STANDARD
        
PTT PUBLIC COMPANY LIMITED
parent: 'PTT PUBLIC COMPANY LIMITED' content: GAS SEPARATION PLANT RAYONG
parent: 'PTT PUBLIC COMPANY LIMITED' content: QUALITY REQUIREMENT FOR
parent: 'PTT PUBLIC COMPANY LIMITED' content: PRESSURE VESSEL
parent: 'PTT PUBLIC COMPANY LIMITED' content: ES-20.02
ES-20.02 PAGE 1 OF 28
parent: 'ES-20.02 PAGE 1 OF 28' content: ES-20.02-D2.docx
QUALITY REQUIREMENTS FOR PRESSURE VESSEL
  
PTT PUBLIC CO., LTD ENGINEERING STANDARD
ES-20.02 PAGE 2 OF 28
parent: 'ES-20.02 PAGE 2 OF 28' content: CONTENTS


1.0,SCOPE
2.0,APPLICABLE CODES
3.0,PTT SPECIFICATION& STANDARD
4.0,DRAWINGS AND RELATED DOCUMENT!
5.0,MATERIALS
6.0,FABRICATION
7.0,TESTING AND INSPECTION
8.0,MARKING


parent: 'CONTENTS' content: APPENDIX 1
parent: 'CONTENTS' content: ES-20.02-D2.docx
parent: 'ES-20.02 PAGE 2 OF 28' content: QUALITY REQUIREMENTS FOR PRESSURE VESSEL
  
PTT PUBLIC CO., LTD ENGINEERING STANDARD
ES-20.02 PAGE 3 OF 28
parent: 'ES-20.02 PAGE 3 OF 28' content: 1.0 SCOPE
parent: '1.0 SCOPE' content: The instructions specified in this standard are the basic, minimum require, and general quality requirements for the design, fabrication, testing and inspection of pressure vessels columns.
parent: '1.0 SCOPE' content: Any conflicting requirements shall be referred to PTT for clarification before proceeding with fabrication of the affected parts.
parent: 'ES-20.02 PAGE 3 OF 28' content: 2.0 APPLICABLE CODES AND STANDARDS.
parent: '2.0 APPLICABLE CODES AND STANDARDS.' content: The American Society of Mechanical Engineers (ASME) Boiler and Pressure Vessel Code (ASME Code)
parent: 'ES-20.02 PAGE 3 OF 28' content: ASME Section II Part A, ferrous Materials


ASME Section V,Non-Destructive Examination
ASME Section VIII Div. 1,Unfired Pressure Vessel
ASME Section VIII Div .2,Alternative rule
ASME Section IX,Welding and Brazing Qualifications


parent: 'ASME Section II Part A, ferrous Materials' content: 2.2 American Society of Mechanical Engineers (ASME)/ American National Standard Institute (ANSI)


...

parent: 'ES-20.02 PAGE 24 OF 28' content: Notes
parent: 'ES-20.02 PAGE 24 OF 28' content: (iv)/(vii )
parent: '(iv)/(vii )' content: Note: 1. (*) PTT’s requirements
parent: '(iv)/(vii )' content: ES-20.02-D2.docx
QUALITY REQUIREMENTS FOR PRESSURE VESSEL
  
PTT PUBLIC CO., LTD ENGINEERING STANDARD
ES-20.02 PAGE 25 OF 28
parent: 'ES-20.02 PAGE 25 OF 28' content: Vessel Class D:


(1),UT is required for all plates with a nominal thickness of 38 mm and above (*).,,
(2),UT is required for all forgings (except for standard nozzle flanges) with a nominal thickness of 100 mm and above (*).,,
(3),MT or PT is required for welding edges of base materials with shell thickness 38 mm and above *),,(iv)
(4),MT or PT is required for backside of double welded joints after being prepared for welding *),,(iv)
(5),"MT of weld surfaces for category A, B, C, D & E *)",,(iv) (vii)
(6),"UT is required for category A,B & D with shell thickness 50 mm and above (*).",,
(7),Charpy impact test is required for all pressure retaining parts.,UCS-66,


parent: 'Vessel Class D:' content: Note: 1. (*) PTT’s requirements
parent: 'Vessel Class D:' content: ES-20.02-D2.docx
QUALITY REQUIREMENTS FOR PRESSURE VESSEL
  
PTT PUBLIC CO., LTD ENGINEERING STANDARD
ES-20.02 PAGE 26 OF 28
parent: 'ES-20.02 PAGE 26 OF 28' content: Vessel Class S:


(1),PT is required for backside of double welded joints after being prepared for welding with shell thickness 25 m and above (*),,(iv)
(2),"PT is required for weld surfaces of: (a) Category A, B, C, D & E in contact with fluids (*). (b) Category A, B, C, D & E when shell thickness 19 mm and above (*). (c) Category C & D when full RT required (*).",,(ix)


parent: 'ES-20.02 PAGE 26 OF 28' content: Note: 1. (*) PTT’s requirements
parent: 'ES-20.02 PAGE 26 OF 28' content: Notes:
parent: 'ES-20.02 PAGE 26 OF 28' content: Main components shall cover such parts as shell, heads, girth, flanges, forged nozzles, tube-sheets, channels and channel covers.
parent: 'ES-20.02 PAGE 26 OF 28' content: The test temperature shall be the minimum design temperature or 0°C, whichever is lower.
parent: 'ES-20.02 PAGE 26 OF 28' content: Charpy impact test shall be according to applicable Codes, Standards and Specifications.
parent: 'ES-20.02 PAGE 26 OF 28' content: This requirement shall apply to the pressure retaining welds (Category A, B & D) in the shell and heads.
parent: 'ES-20.02 PAGE 26 OF 28' content: Category A & B in nozzles that are not subject to RT shall additionally be examined by UT.
parent: 'ES-20.02 PAGE 26 OF 28' content: vi) When full RT required, category A & B in nozzles that are not suitable to RT shall be examined by UT.
parent: 'ES-20.

Here we clearly see that Unstructured is parsing both table and document structure.

### Storing into Astra DB

Now we will continue with the RAG process, by creating embeddings for the pdf, and storing them in Astra.

In [35]:
from langchain_astradb import AstraDBVectorStore
from langchain_openai import OpenAIEmbeddings

astra_db_store = AstraDBVectorStore(
    collection_name="PTT_unstructured",
    embedding=OpenAIEmbeddings(),
    token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
    api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT")
)

We will create LangChain Documents by splitting the text after `Table` elements and before `Title` elements. Additionally, we use the html output format for table data.

In [36]:
from langchain_core.documents import Document

documents = []
current_doc = None

for el in elements:
    if el.category in ["Header", "Footer"]:
        continue # skip these
    if el.category == "Title":
        documents.append(current_doc)
        current_doc = None
    if not current_doc:
        current_doc = Document(page_content="", metadata=el.metadata.to_dict())
    current_doc.page_content += el.metadata.text_as_html if el.category == "Table" else el.text
    if el.category == "Table":
        documents.append(current_doc)
        current_doc = None



In [37]:
# Sanitise the data by filtering out None values from the documents list
documents = [doc for doc in documents if doc is not None]


In [38]:
astra_db_store.add_documents(documents)

['3dd3456e271747ebad9ad986ae8a6982',
 '8caf451b19474519b13e0fc0660668de',
 'e33730ebd3bf4ba7aa3ad2807c225f4c',
 '192062b51cf64965a113e98a442f32f8',
 '09d4864357744352a3ac3de4151de443',
 '1f83710dbb104c5b917f491b2cf2b087',
 'f44214f3b8844974a43d31d75106ad6d',
 '1c20aa1592ac4f9c971faf1097290ebd',
 'f60f8b27d90e4aa5bea8aa951f2cd6b1',
 '99d6376b057b4ce79623302788b04e77',
 '7d4bb9d049874dfca20d78839218f3bd',
 'e1f2734a0c7a4065acf2322e5cd6551d',
 'a76865a4cb7b412d84e2d5c83423e73b',
 'd93602adfe2c47a493305dd3b8b18e50',
 'c3cee894dc494a6b9d70e153e118f1fc',
 '8a3222714e9640019e64b190082d6e7d',
 '7ad2c73727ce4821b4198c5e3bf2d55b',
 'bfcfba34e7d74bffb4b6d0790203c317',
 '6141c30088534d35aa6e8a4eda6b35ee',
 '29dc355c202642f6ba713cd8ac49f13c',
 'd31854c3536146948542e267ca8c314f',
 'ab488d7751f44941b779eb1c3e20391b',
 'f1b78250733d41048631418ff4b503fd',
 'd45b8f7429a5454fb484bf1ab53cf39d',
 'b5e047a7e4a34fdd8305495f16b73a1f',
 '170cbcbf653b4d07ab2c461afa02789a',
 '912da3df14f342e49e0cabb96876af16',
 

### Querying

Now that we have populated our vector store, we will build a RAG pipeline and execute some queries.

In [39]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

prompt = """
Answer the question based only on the supplied context. If you don't know the answer, say "I don't know".
Context: {context}
Question: {question}
Your answer:
"""

llm = ChatOpenAI(model="gpt-3.5-turbo", streaming=False, temperature=0)

chain = (
    {"context": astra_db_store.as_retriever(), "question": RunnablePassthrough()}
    | PromptTemplate.from_template(prompt)
    | llm
    | StrOutputParser()
)

First we can ask a question about some text in the document:

In [40]:
chain.invoke("What kind of markings should be used?")

'Markings in ink, oily paint, or waterproof color should be used.'

Next we can try to get a value from the 2nd table:

In [ ]:
chain.invoke("What should be the marking symbol for TYPE 304L S.S?")

'S304L'

In [42]:
chain.invoke("What is the recommended heat treatment for cold forming SS material")

'Annealing, normalizing, and tempering as required by table 1 should be performed in accordance with table 2 for cold forming stainless steel material.'

In [44]:
 chain.invoke("What is the holding temperature for post weld heat treatment for Carbon steel material type?")

"I don't know."

In [49]:
#Debug
astra_db_store.similarity_search_with_score("Carbon steel")

[(Document(page_content='1. CLASSIFICATION OF THE VESSELVessels and their parts are classified as shown in table A:Table A. Classification of Vessel<table><thead><th>Material and Services Carbon steel for high and intermediate temperature service (Design temp.&gt;- 10°C</th><th>Killed Steel</th><th>Class</th></thead><tr><td>Carbon steel for low temperature service (Design temp. &lt;-10°C)</td><td>Fine Grained steel</td><td>C</td></tr><tr><td>Low-alloy steel for low temperature service</td><td>2.5Ni 3.5 Ni Killed steel</td><td>D</td></tr><tr><td>Austenitic stainless steel</td><td></td><td></td></tr></table>', metadata={'filetype': 'application/pdf', 'languages': ['eng'], 'page_number': 21, 'parent_id': '674ecd15c17e6673137a36e51247bf34', 'filename': 'ES-20.02-D2_Quality_Requirement_for_Pressure_Vessel.pdf'}),
  0.91858506),
 (Document(page_content='PTT PUBLIC CO., LTD ENGINEERING STANDARD<table><thead><th>Carbon steel</th><th>600+/-20</th><th>60</th></thead><tr><td>1.25Cr-0.5Mo</td><td>

In [ ]:
#Debug
astra_db_store.similarity_search_with_score("What is the holding temperature for post weld heat treatment for Carbon steel material type?")

[(Document(page_content='6.3 Post weld Heat Treatment( PWHT)6.3.1 PWHT shall not be commenced unless and until all non-destructive testing has been successfully completed. PWHT shall be performed prior final examination.6.3.2 No welding is permissible on the equipment after completion of post weld heat treatment.6.3.3 All carbon steel and C- Mn steels shall be subject to PHWT where “Sour service” or “Amine service” is indicated.6.3.4 A continuous record of temperature shall be made on recorder charts.6.3.5 The holding temperature for post weld heat treatment or stress relief temperature shall be :', metadata={'filetype': 'application/pdf', 'languages': ['eng'], 'page_number': 11, 'parent_id': '917e2a0ba25081f5662f26efa4039865', 'filename': 'ES-20.02-D2_Quality_Requirement_for_Pressure_Vessel.pdf'}),
  0.942477),
 (Document(page_content='6.3.6 When clad steel or dissimilar welded parts are heat-treated, the heat- treating procedure shall be submitted for PTT /CONSULTANT approval.Austeni

And finally we can ask a question that doesn't exist in our content to confirm that the LLM rejection is working correctly.